<a href="https://www.kaggle.com/code/isaacbernadus/tubes?scriptVersionId=138126890" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
BATCH_SIZE = 32
IMG_HEIGHT = 512
IMG_WIDTH = 512

data_dir = "../input/d/isaacbernadus/batik-data-set/training_files_cleaned"
data_dir = pathlib.Path(data_dir)
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  color_mode="grayscale",
  shuffle=True,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  crop_to_aspect_ratio=True)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  color_mode="grayscale",
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  crop_to_aspect_ratio=True)


class_names = train_ds.class_names
print(class_names)



# Check for any files that is incompatible with TensorFlow

In [ ]:
import pathlib
from pathlib import Path
import imghdr

image_extensions = [".png",".jpg"]  # add there all your images file extensions
img_type_accepted_by_tf = ["jpeg","jpg"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

# Tune and build model

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

num_classes = len(class_names)

data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(IMG_HEIGHT,
                                  IMG_WIDTH,
                                  1)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
  layers.Conv2D(8, 3, padding='same', activation='relu'),
  layers.Dropout(0.2),
  layers.MaxPooling2D(),

  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.Dropout(0.2),
  layers.MaxPooling2D(),

  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Dropout(0.2),
  layers.MaxPooling2D(),

  layers.Flatten(),
  layers.Dropout(0.5),
  layers.Dense(64, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(num_classes, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

# Train Model

In [ ]:
epochs=130
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


# Evaluate Model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


# Test Model with test_files

In [ ]:
test_directory = "../input/d/isaacbernadus/batik-data-set/test_files"
files = pathlib.Path(test_directory).glob('*')

print ("{:<50} {:<20} {:<30} {:<20} {:<20}".format('File','Kawung','Mega Mendung','Parang', 'Truntum'))
for file in files:
#     print(os.path.basename(file))
    img = tf.keras.utils.load_img(
    file,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode="grayscale"
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    print ("{:<50} {:<20} {:<30} {:<20} {:<20}".format(os.path.basename(file), 
                                 str(round(predictions[0,0]*100,2)), 
                                 str(round(predictions[0,1]*100,2)),
                                 str(round(predictions[0,2]*100,2)),
                                 str(round(predictions[0,3]*100,2))))

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

In [ ]:
import pathlib
import tensorflow as tf
import numpy as np
model2 = tf.keras.models.load_model('saved_model/my_model')

test_directory = "../input/d/isaacbernadus/batik-data-set/test_files"
files = pathlib.Path(test_directory).glob('*')

class_name = [
    "Kawung",
    "Mega_Mendung",
    "Parang",
    "Truntum"
]

kawung_test = pathlib.Path(test_directory).glob('*kawung*')
truntum_test = pathlib.Path(test_directory).glob('*truntum*')
parang_test = pathlib.Path(test_directory).glob('*parang*')
mega_mendung_test = pathlib.Path(test_directory).glob('*mega_mendung*')
test_sum = [0,0,0,0]
for file in kawung_test:
    test_sum[0] += 1

for file in mega_mendung_test:
    test_sum[1] += 1
    
for file in parang_test:
    test_sum[2] += 1

for file in truntum_test:
    test_sum[3] += 1
    
val_sum = [0,0,0,0]
val_sum_true = [0,0,0,0]
val_sum_true_neg = [0,0,0,0]
print("PENGENAL CORAK BATIK")
print("Sampel: training_files_b")
print("Metode: CNN dengan Tensorflow dan Multilabel Classification")

print("------")
print ("{:<50} {:<10} {:<20} {:<10} {:<10}".format('File','Kawung','Mega Mendung','Parang', 'Truntum'))
for file in files:
#     print(os.path.basename(file))
    img = tf.keras.utils.load_img(
    file,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode="grayscale"
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model2.predict(img_array)
    filename = os.path.basename(file)
    
    # True Positive
    if class_name[np.argmax(predictions)] == "Kawung" and filename.find("kawung") !=-1 :
        val_sum_true[0] += 1
    elif class_name[np.argmax(predictions)] == "Mega_Mendung" and filename.find("mega_mendung") !=-1:
        val_sum_true[1] += 1
    elif class_name[np.argmax(predictions)] == "Parang" and filename.find("parang")!=-1:
        val_sum_true[2] += 1
    elif class_name[np.argmax(predictions)] == "Truntum" and filename.find("truntum")!=-1:
        val_sum_true[3] += 1
        
    # False Positive
    if class_name[np.argmax(predictions)] == "Kawung" and filename.find("kawung") ==-1 :
        val_sum[0] += 1
    elif class_name[np.argmax(predictions)] == "Mega_Mendung" and filename.find("mega_mendung") ==-1 :
        val_sum[1] += 1
    elif class_name[np.argmax(predictions)] == "Parang" and filename.find("parang") == -1 :
        val_sum[2] += 1
    elif class_name[np.argmax(predictions)] == "Truntum" and filename.find("truntum") == -1:
        val_sum[3] += 1
        
    #False Negative
    val_sum_false_neg=[0,0,0,0]
    
    val_sum_false_neg[0] = test_sum[0] - val_sum_true[0]
    val_sum_false_neg[1] = test_sum[1] - val_sum_true[1]
    val_sum_false_neg[2] = test_sum[2] - val_sum_true[2]
    val_sum_false_neg[3] = test_sum[3] - val_sum_true[3]
    
    #True Negative
    if filename.find("kawung") ==-1 and class_name[np.argmax(predictions)] != "Kawung" :
        val_sum_true_neg[0] += 1
    elif filename.find("mega_mendung") ==-1 and class_name[np.argmax(predictions)] != "Mega_Mendung":
        val_sum_true_neg[1] += 1
    elif filename.find("parang") ==-1 and class_name[np.argmax(predictions)] != "Parang":
        val_sum_true_neg[2] += 1
    elif  filename.find("truntum") ==-1 and class_name[np.argmax(predictions)] != "Truntum":
        val_sum_true_neg[3] += 1
    
    print ("{:<50} {:<10} {:<20} {:<10} {:<10}".format(os.path.basename(file), 
                                 str(round(predictions[0,0]*100,2)), 
                                 str(round(predictions[0,1]*100,2)),
                                 str(round(predictions[0,2]*100,2)),
                                 str(round(predictions[0,3]*100,2))))
    

print("")
print ("{:<50} {:<10} {:<20} {:<10} {:<10} {:<10}".format('Kinerja','Kawung','Mega Mendung','Parang', 'Truntum', "TOTAL"))
print("------")
print ("{:<50} {:<10} {:<20} {:<10} {:<10} {:<10}".format('True-Positif', val_sum_true[0], val_sum_true[1], val_sum_true[2], val_sum_true[3], sum(val_sum_true)))
print ("{:<50} {:<10} {:<20} {:<10} {:<10} {:<10}".format('False-Positif', val_sum[0], val_sum[1], val_sum[2], val_sum[3], sum(val_sum)))
print ("{:<50} {:<10} {:<20} {:<10} {:<10} {:<10}".format('True-Negative', val_sum_true_neg[0], val_sum_true_neg[1], val_sum_true_neg[2], val_sum_true_neg[3], sum(val_sum_true_neg)))
print ("{:<50} {:<10} {:<20} {:<10} {:<10} {:<10}".format('False-Negative', val_sum_false_neg[0], val_sum_false_neg[1], val_sum_false_neg[2], val_sum_false_neg[3], sum(val_sum_false_neg)))


accuracy=(sum(val_sum_true) + sum(val_sum_true_neg))/(sum(val_sum_true) + sum(val_sum_true_neg) + sum(val_sum) + sum(val_sum_false_neg))
precision = sum(val_sum_true) / (sum(val_sum_true) + sum(val_sum))
sensitivity = sum(val_sum_true)/ (sum(val_sum_true) + sum(val_sum_false_neg))
specificity = sum(val_sum_true_neg) / (sum(val_sum_true_neg) + sum(val_sum_false_neg))
f1_score = (2*sensitivity*precision)/(sensitivity + precision)
print("-----")
print("{:<50} {:<20}".format("Accuracy (%)", accuracy*100))
print("{:<50} {:<20}".format("Precision (%)", precision*100))
print("{:<50} {:<20}".format("Sensitivity (%)", sensitivity*100))
print("{:<50} {:<20}".format("Specificity (%)", specificity*100))
print("{:<50} {:<20}".format("F1 Score (%)", f1_score*100))


In [ ]:
# accuracy = int((sum(val_sum_true)+sum(val_sum_true_neg))/(sum(val_sum_true)+sum(val_sum_true_neg+sum(val_sum)+sum(val_sum_false_neg))))
print(accuracy)